## Processing Ocean Color Imagery - Process Micasense Images

Pull in all images and process with micasense tools
    normalize and align and whatnot
    put these into functions and clean up
    also visualize along the way and look at distributions to think about whether or not they're legitimate
    write out
    
Pull in all images as modified and metadata with specific functions
    Visualize for issues and shadows and bright spots and locations
    correct for sky radiance
    visualize the distributions of both sky (clouds) and sea
    look at the variability of spectra both for specific pixels and across nearby images
    filter for orientation and location and issues

take the water leaving spectra and convert to chla as well as other future products and just intro general analysis of spectra
    produce a csv or .shp or images of ocean color products 
    

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ipywidgets import FloatProgress, Layout
from IPython.display import display
from micasense import imageset as imageset
from micasense import capture as capture
import multiprocessing
import glob, shutil, os

import cv2
import numpy as np
import matplotlib.pyplot as plt


import micasense.imageutils as imageutils
import micasense.plotutils as plotutils

import exiftool
import datetime

import subprocess
import math

from utils import *
from georeg import *

### Run the process

#### first sort the images into sea and sky

In [3]:
! ls data/20210407_duml_full_workflow_test

all_imgs       sea	    sky_rededge
panel	       sea_rededge  sunblocked_rededge
panel_rededge  sky	    SYNLog-13-24-55_07-04-2021.csv


In [4]:
all_img_dir = os.path.join('data', '20210427_gulf_stream_surveys', 'flight1', 'all_imgs')
sky_dst = os.path.join('data', '20210427_gulf_stream_surveys', 'flight1', 'sky')
sea_dst = os.path.join('data', '20210427_gulf_stream_surveys', 'flight1', 'sea')

In [5]:
if not os.path.exists(sky_dst):
    os.makedirs(sky_dst)

if not os.path.exists(sea_dst):
    os.makedirs(sea_dst)

down_image_count = 10
up_image_count = 2

#go through each image
i = 0
j = 0
count = 0
for file in sorted(glob.glob(all_img_dir+ "/*.tif")):
    if i < down_image_count*6:
        shutil.move(file, sea_dst)
        i += 1
        if i == down_image_count*6:
            j = 0
    elif j < up_image_count*6:
        shutil.move(file, sky_dst)
        j += 1
        if j == up_image_count*6:
            i = 0
    count += 1
print('moved', count, 'images')

moved 3234 images


### This is the typical full workflow

In [6]:
glob.glob(os.path.join('data', '20210427_gulf_stream_surveys', 'flight1', 'panel', 'IMG_0027*.tif'))

['data/20210427_gulf_stream_surveys/flight3/panel/IMG_0027_4.tif',
 'data/20210427_gulf_stream_surveys/flight3/panel/IMG_0027_3.tif',
 'data/20210427_gulf_stream_surveys/flight3/panel/IMG_0027_6.tif',
 'data/20210427_gulf_stream_surveys/flight3/panel/IMG_0027_2.tif',
 'data/20210427_gulf_stream_surveys/flight3/panel/IMG_0027_1.tif',
 'data/20210427_gulf_stream_surveys/flight3/panel/IMG_0027_5.tif']

In [7]:
! ls data/20210427_gulf_stream_surveys/flight1

all_imgs  chla	panel  rededge	sea  sky


#### First process the Altum images

From https://support.micasense.com/hc/en-us/articles/360025336894-Using-Panels-and-or-DLS-in-Post-Processing

In the case of a clear, sunny day, we recommend using only the calibration panels. Light sensor data will not be as effective and so should generally not be used in this case.

Light sensor data is most effective in overcast, completely cloudy conditions. In this case, we would recommend using both light sensor and panel data for calibration.

In [10]:
# to use the DLS and not the panel set this to None
#panelNames = None
panelNames = glob.glob(os.path.join('data', '20210427_gulf_stream_surveys', 'flight1', 'panel', 'IMG_0027*.tif'))
# sky_dir = os.path.join('data', '20210427_gulf_stream_surveys', 'flight1', 'sky')
# warp_img_dir = os.path.join('data', '20210427_gulf_stream_surveys', 'flight1', 'sky')
# sky_out = process_micasense_subset(sky_dir, panelNames, warp_img_dir, overwrite=False)

surface_dir = os.path.join('data', '20210427_gulf_stream_surveys', 'flight1', 'sea')
warp_img_dir = os.path.join('data', '20210427_gulf_stream_surveys', 'flight1', 'sea', 'align_img')
surface_out = process_micasense_subset(surface_dir, panelNames, warp_img_dir, overwrite=False)

Panel irradiance calculated.
used warp dir data/20210427_gulf_stream_surveys/flight3/sea/align_img
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 0
Finished aligning band 2
Finished aligning band 4
Finished aligning band 1
Finished aligning band 3
Saving time: 0:23:52.542259
Alignment+Saving rate: 0.31 images per second
Finished saving images.
exiftool -csv="data/20210427_gulf_stream_surveys/flight3/sea/stacks/log.csv" -overwrite_original data/20210427_gulf_stream_surveys/flight3/sea/stacks
Finished saving image metadata.


In [9]:
! exiftool -csv="data/20210427_gulf_stream_surveys/flight1/sky/stacks/log.csv" -overwrite_original data/20210427_gulf_stream_surveys/flight3/sky/stacks

    1 directories scanned
   89 image files updated


In [11]:
! exiftool -csv="data/20210427_gulf_stream_surveys/flight1/sea/stacks/log.csv" -overwrite_original data/20210427_gulf_stream_surveys/flight3/sea/stacks

    1 directories scanned
  449 image files updated


#### Now process the RedEdge Images

Need to ensure these are the correct images via time. They do seem correct.

Also note that the first couple sea images were bad because of spray and glint and shadow.

In [19]:
! ls data/20210427_gulf_stream_surveys/flight1/rededge/

panel  sea  sky  tube


In [20]:
# to use the DLS and not the panel set this to None
#panelNames = None
panelNames = glob.glob(os.path.join('data', '20210427_gulf_stream_surveys/flight1', 'rededge', 'panel', 'IMG_0001*.tif'))

blocked_dir = os.path.join('data', '20210427_gulf_stream_surveys/flight1', 'rededge', 'tube')
sky_dir = os.path.join('data', '20210427_gulf_stream_surveys/flight1', 'rededge', 'sky')
surface_dir = os.path.join('data', '20210427_gulf_stream_surveys/flight1', 'rededge', 'sea')

warp_img_dir = os.path.join('data', '20210427_gulf_stream_surveys/flight1', 'rededge', 'sea')
sunlight_blocked_out = process_micasense_subset(blocked_dir, panelNames, warp_img_dir, overwrite=False)

warp_img_dir = os.path.join('data', '20210427_gulf_stream_surveys/flight1', 'rededge', 'sky')
sky_out = process_micasense_subset(sky_dir, panelNames, warp_img_dir, overwrite=False)

warp_img_dir = os.path.join('data', '20210427_gulf_stream_surveys/flight1', 'rededge', 'sea')
surface_out = process_micasense_subset(surface_dir, panelNames, warp_img_dir, overwrite=False)

Panel irradiance calculated.
used warp dir data/20210427_gulf_stream_surveys/flight3/rededge/sea
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 0
Finished aligning band 3
Finished aligning band 1
Finished aligning band 4
Finished aligning band 2
Saving time: 0:00:27.324249
Alignment+Saving rate: 0.73 images per second
Finished saving images.
exiftool -csv="data/20210427_gulf_stream_surveys/flight3/rededge/tube/stacks/log.csv" -overwrite_original data/20210427_gulf_stream_surveys/flight3/rededge/tube/stacks
Finished saving image metadata.
Panel irradiance calculated.
used warp dir data/20210427_gulf_stream_surveys/flight3/rededge/sky
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 0
Finished aligning band 3
Finished aligning band 4
Finished aligning band 1
Finished aligning band 2
Saving time: 0:00:11.340801
Alignment+Saving rate: 0.88 images per second
Finished

In [21]:
! exiftool -csv="data/20210427_gulf_stream_surveys/flight1/rededge/tube/stacks/log.csv" -overwrite_original data/20210427_gulf_stream_surveys/flight3/rededge/tube/stacks


    1 directories scanned
   20 image files updated


In [22]:
! exiftool -csv="data/20210427_gulf_stream_surveys/flight1/rededge/sky/stacks/log.csv" -overwrite_original data/20210427_gulf_stream_surveys/flight3/rededge/sky/stacks


    1 directories scanned
   10 image files updated


In [23]:
! exiftool -csv="data/20210427_gulf_stream_surveys/flight1/rededge/sea/stacks/log.csv" -overwrite_original data/20210427_gulf_stream_surveys/flight3/rededge/sea/stacks


    1 directories scanned
   14 image files updated
